## 標準モジュールでのスクレイピング

### ライブラリの読み込み

In [1]:
from urllib.request import urlopen
from html import unescape
import numpy as np
import re
import sys
import feedparser
import json

### webページの読み込み

In [2]:
f = urlopen("http://sample.scraping-book.com/dp")
bytes_content = f.read()
print(f.getheader("Content-Type"))
scanned_text = bytes_content.decode("UTF-8", errors="replace")
print(scanned_text[:1024])

text/html; charset=utf-8
<!DOCTYPE HTML>
<html lang="ja" class="pc">
<head>
  <meta charset="UTF-8">
  <title>Pythonクローリング＆スクレイピング -データ収集・解析のための実践開発ガイド- サンプルページ</title>
  <meta http-equiv="Content-Style-Type" content="text/css"/>
  <meta http-equiv="Content-Script-Type" content="application/javascript"/>
  <meta name="description" content="Pythonクローリング＆スクレイピング -データ収集・解析のための実践開発ガイド- サンプルページ"/>
  <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"/>
  <meta name="apple-mobile-web-app-capable" content="yes"/>
  <meta name="format-detection" content="telephone=no"/>
  <link rel="shortcut icon" href="/assets/templates/gdp/favicon.ico" type="image/vnd.microsoft.icon"/>
  <meta name="robots" content="noindex,noarchive"/>
  <link rel="stylesheet" href="/dp/assets/style/store1124.css" type="text/css" media="all"/>
</head>
<body itemscope="itemscope" itemtype="http://schema.org/WebPage" class="page-home">
  <header id="header" itemscope="itemscope" itemtype="http://schema.org/WPHeader"

### 正規表現の例

In [3]:
print(re.search("a", "abc123DEF"))
print(re.search("a.*d", "abc123DEF", re.IGNORECASE))
print(re.search("a(.*)c", "abc123DEF"))
print(re.findall("\w{2,}", "This is a pen"))
print(re.sub("\w{2,}", "That", "This is a pen"))

<_sre.SRE_Match object; span=(0, 1), match='a'>
<_sre.SRE_Match object; span=(0, 7), match='abc123D'>
<_sre.SRE_Match object; span=(0, 3), match='abc'>
['This', 'is', 'pen']
That That a That


### urlと書籍タイトルを取得する

In [5]:
#データの格納用配列
text = re.findall(r'<a itemprop="url".*?</a></li>', scanned_text, re.DOTALL)   #正規表現で文章を分離
url_list = np.array(["" for i in range(len(text))], dtype="U65536")
title_list = np.array(["" for i in range(len(text))], dtype="U65536")

for j in range(len(text)):
    #urlを取得
    url = re.search('<a itemprop="url" href="(.*?)">', text[j])
    if not url:
        continue
    url = url.group(1)
    url_list[j] = 'https://gihyo.jp' + url

    #タイトルを取得
    title = re.search('<p itemprop="name".*?</p>', text[j])
    if not title:
        continue
    title = title.group(0)
    title = title.replace('<br/>', '')
    title = re.sub('<.*?>', '', title)
    title_list[j] = unescape(title)
    
#取得したデータを確認
print(title_list)
print(url_list)

['これからはじめるプログラミング 作って覚える基礎の基礎'
 '情報処理技術者試験 平成28年度【秋期】情報セキュリティマネジメント パーフェクトラーニング過去問題集'
 '大人の自由時間（大人の自由時間mini） 水泳のきれいなカラダをつくる～スリムな逆三角形になる！ドライランドトレーニング'
 '30レッスン 30レッスンで絶対合格！ Microsoft Office Specialist PowerPoint 2013 テキスト＋問題集'
 'ゼロからはじめる ゼロからはじめる海外旅行でスマホ活用 スマートガイド'
 '今すぐ使えるかんたんmini 今すぐ使えるかんたんminiCD&DVD 作成超入門［Windows 10対応版］'
 '大きな字でわかりやすい 大きな字でわかりやすいワード2016入門' '大きな字でわかりやすい 大きな字でわかりやすいエクセル2016 入門'
 '今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ デジカメ写真活用術［Windows 10対応版］'
 'Software Design 2016年8月号'
 '無料ではじめるBlender CGイラストテクニック～3DCGの考え方としくみがしっかりわかる'
 '小さなお店＆会社の WordPress超入門―初めてでも安心！思いどおりのホームページを作ろう！'
 'WEB+DB PRESS plus Atom実践入門──進化し続けるハッカブルなエディタ'
 'アウトライナー実践入門～「書く・考える・生活する」創造的アウトライン・プロセッシングの技術〜'
 'ゼロからはじめる ゼロからはじめるドコモGalaxy S7 edge SC-02H スマートガイド'
 'Excelのムカムカ！が一瞬でなくなる使い方～表計算・資料作成のストレスを最小限に！'
 'Wordのムカムカ！が一瞬でなくなる使い方～文章・資料作成のストレスを最小限に！'
 '生物ミステリー（生物ミステリー プロ） そもそも島に進化あり'
 '大人の自由時間（大人の自由時間mini） 山歩きスタートブック～道具と歩き方がわかる，行きたいコースが見つかる'
 '平成28-29年度 基本情報技術者 試験によくでる問題集【午後】'
 '大人の自由時間（大人の自由時間mini） あきらめないランニング～楽しいランのは

### XML(RSS)のスクレイピング

In [7]:
#rssを取得
rss = feedparser.parse('http://gihyo.jp/feed/rss2', response_headers={"content-type": "text/xml; charset=utf-8"})

In [8]:
#データの格納用配列
url_list = np.array(["" for i in range(len(rss.entries))], dtype="U65536")
title_list = np.array(["" for i in range(len(rss.entries))], dtype="U65536")
text = rss.entries

#タイトルとurlを取得
for j in range(len(text)):
    #urlを取得
    url = text[j].link
    if not url:
        continue
    url_list[j] = url

    #タイトルを取得
    title = text[j].title
    if not title:
        continue
    title_list[j] = title
print(title_list)
print(url_list)

['2019年3月2日\u3000鳴くねこ ── 帰ってきた『技評ねこ部通信』'
 '1,000名を超すAndroidに関わる開発者が集まった DroidKaigi 2019参加レポート ── レポート'
 '第43回\u3000MySQL 8.0.15緊急リリースとLINEでの導入事例公開，PostgreSQL 11.2，Pgpool-II 4.0.3リリース ── OSSデータベース取り取り時報'
 "WEB+DB PRESS Vol.108へのお便り ── はまちちゃんとわかばちゃんのREADER'S FORUM―読者のページ"
 'ネットギアジャパン合同会社，Amazon Alexa付属「Orbi Voice メッシュWiFiシステム」が日本初上陸，本日より予約開始 ── ニュースリリース'
 '2019年2月第4週\u3000Galaxy S10e，S10，S10+，そして，Galaxy Foldを発表 ── Android Weekly Topics'
 '2019年2月第5週号 1位は，デザインシステムを知るヒントとして実例を紹介，気になるネタは，"アルバム"の老舗，無料の容量無制限画像保存クラウドストレージサービスをローンチ ── 週刊Webテク通信'
 '第558回\u3000LibreOffice 6.2のユーザーインターフェース概要 ── Ubuntu Weekly Recipe'
 '第16回\u3000BBc-1 今後の展望 ── ブロックチェーンの課題と可能性～BBc-1（Beyond Blockchain One）から学ぶブロックチェーン開発'
 '第7回\u3000ウィスキー・ボトル・ブルース（1番） ── 続・玩式草子 ―戯れせんとや生まれけん―'
 '第91回\u3000テーブルごとのリクエストやI/Oの統計を確認する ── MySQL道普請便り'
 '2019年2月25日\u3000"急ぐ必要はない" ―Linus，「Linux 5.0-rc8」を公開，正式リリースは3月3日に ── Linux Daily Topics'
 '最終回\u3000Android 端末におけるライブ配信サービスの実現（前編） ── 大人気動画コミュニティアプリの運用の内幕―MixChannel（ミクチャ）を支える技術'
 '第4回\u3000YAP

'2019年3月2日\u3000鳴くねこ ── 帰ってきた『技評ねこ部通信』'